In [19]:
!pip install panel

  Obtaining dependency information for panel from https://files.pythonhosted.org/packages/24/5d/d0f66d05f20068b40df46b84bc68054f6fce5e458d58e9a9585eb0d601e2/panel-1.3.8-py2.py3-none-any.whl.metadata
  Obtaining dependency information for bokeh<3.4.0,>=3.2.0 from https://files.pythonhosted.org/packages/39/ba/aefd7aacc9e086e2c7f3bb42e99cb8e2a8f24dcb4bf78519ef25a9102988/bokeh-3.3.4-py3-none-any.whl.metadata
  Obtaining dependency information for param<3.0,>=2.0.0 from https://files.pythonhosted.org/packages/d7/04/d2b3c627bc9ea4477affca78f71bcadb84d73873f133ce4c47a03ebaaf59/param-2.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for pyviz-comms>=2.0.0 from https://files.pythonhosted.org/packages/88/e1/a877ce6f15fcf51da1ef0a253266c3f321bec7ed6f93cc82e8a8296c8618/pyviz_comms-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for xyzservices>=2021.09.1 from https://files.pythonhosted.org/packages/82/c3/e06dfa46464cce3eda4b86df8847cab99d9bc545c76807ee689545187a


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import glob
import os
import pickle

In [2]:
with open('D:\Weather Project\data\MET Data\complete_data.pkl', 'rb') as file:
    df = pickle.load(file)

df = df[['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)','REGIAO', 'UF', 'ESTACAO',
       'CODIGO (WMO)']]

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_8512\1666682795.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\complete_data.pkl', 'rb') as file:


In [3]:
with open('D:\Weather Project\data\MET Data\glossary.pkl', 'rb') as file:
    glossary = pickle.load(file)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_8512\173438111.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\glossary.pkl', 'rb') as file:


In [4]:
# Drop cities with only missing values
nan_percentages_by_city = df.groupby('CODIGO (WMO)')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].apply(lambda x: (x.isna().sum() / len(x)))
missing_idxs = nan_percentages_by_city[nan_percentages_by_city> 0.95].index
df = df[~df['CODIGO (WMO)'].isin(missing_idxs)]

In [5]:
df['lag_precip'] = df.groupby('CODIGO (WMO)')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].shift(1)
df['rain_start_count'] = 0
df.loc[(df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] > 0) & (df['lag_precip']==0), 'rain_start_count'] = 1

In [6]:
city_week_group = df.groupby(['CODIGO (WMO)', pd.Grouper(freq='W', key='Data')])[['rain_start_count']].sum()
city_week_group.reset_index(inplace=True)

In [7]:
city_group = city_week_group.groupby('CODIGO (WMO)').mean(numeric_only=True)
city_group.rename(columns={
    'rain_start_count':'mean weekly rain starts'
}, inplace =True)

city_group = city_group.merge(glossary, left_index=True, right_index=True)

In [8]:
# Create a new column with the sorted order of 'Column1' in descending order
city_group['Precip Raning'] = (-city_group['mean weekly rain starts']).argsort().argsort()

In [17]:
city_group[city_group['ESTACAO']=='CURITIBA']

,mean weekly rain starts,REGIAO,UF,ESTACAO,LATITUDE,LONGITUDE,ALTITUDE,Precip Raning
CODIGO (WMO),,,,,,,,
A807,6.280255,S,PR,CURITIBA,"-25,4486111","-49,23055554","922,91",38


In [18]:
city_group[city_group['UF']=='SC']

,mean weekly rain starts,REGIAO,UF,ESTACAO,LATITUDE,LONGITUDE,ALTITUDE,Precip Raning
CODIGO (WMO),,,,,,,,
A806,6.681529,S,SC,FLORIANOPOLIS,"-27,60253","-48,620096","4,87",33
A814,6.789809,S,SC,URUSSANGA,"-28,53249999","-49,31527777","40,56",30
A815,7.235669,S,SC,SAO JOAQUIM,"-28,27564","-49,934617","1400,06",23
A816,3.152866,S,SC,NOVO HORIZONTE,"-26,40638888","-52,85027777","943,57",281
A817,4.350318,S,SC,INDAIAL,"-26,91361111","-49,26805554","72,24",161
A841,6.000000,S,SC,JOACABA,"-27,16916666","-51,55888888","767,63",47
A845,9.777070,S,SC,BOM JARDIM DA SERRA - MORRO DA IGREJA,"-28,126992","-49,47961","1790,38",2
A848,4.738854,S,SC,DIONISIO CERQUEIRA,"-26,286562","-53,633114","807,54",118
A851,6.286624,S,SC,ITAPOA,"-26,08138888","-48,64166666","6,18",37


In [10]:
city_group.sort_values('mean weekly rain starts', ascending=False)

,mean weekly rain starts,REGIAO,UF,ESTACAO,LATITUDE,LONGITUDE,ALTITUDE,Precip Raning
CODIGO (WMO),,,,,,,,
A201,10.191083,N,PA,BELEM,"-1,411228","-48,439512","21,17",0
A401,9.834395,NE,BA,SALVADOR,"-13,005515","-38,50576","47,56",1
A845,9.777070,S,SC,BOM JARDIM DA SERRA - MORRO DA IGREJA,"-28,126992","-49,47961","1790,38",2
A870,9.605096,S,SC,RANCHO QUEIMADO,"-27,6786111","-49,04194444",881,3
A624,9.178344,SE,RJ,NOVA FRIBURGO - SALINAS,"-22,33472221","-42,67694443",1070,4
...,...,...,...,...,...,...,...,...
A318,0.152866,NE,RN,MOSSORO,"-4,90416666","-37,36694443","29,44",560
A317,0.076433,NE,RN,MACAU,"-5,15111111","-36,57305554","17,37",561
A443,0.070064,NE,BA,DELFINO,"-10,455","-41,20694444","646,24",562
